<a href="https://colab.research.google.com/github/jessicagohh/ISO-3166-Countries-with-Regional-Codes/blob/master/rider_churn_draftcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code here does not reflect the intended production-ready setup. 

It fulfils the following purposes:

1. Provide a logic template for retrieving and preparing data, and using that to train a PBND model.
2. Produce a model for Argentina riders, for internal testing.

In [2]:
!pip install --upgrade gspread
!pip install pandas
!pip install numpy
!pip install lifetimes
!pip install pandas_gbq
!pip install pydata_google_auth

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1
     |████████████████████████████████| 593kB 6.0MB/s 


In [3]:
# imports

import pandas_gbq
import pydata_google_auth
import pandas as pd
import numpy as np

from lifetimes.utils import summary_data_from_transaction_data
from lifetimes import ParetoNBDFitter

from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials


In [ ]:
# loads the data required
project_id = "fulfillment-dwh-production"

"""enter variables here"""
input_threshold_date = 'DATE(2020,8,1)'
input_end_date = 'DATE(2020,10,31)'


sql = """
WITH date_array AS (
SELECT
GENERATE_DATE_ARRAY( {threshold_date}, {end_date} ) as dates_column
)

, report_dates AS(
SELECT 
report_date

FROM date_array, UNNEST(dates_column) as report_date
)

, rider_list AS(
SELECT
country_code
, rider_id
, CONCAT( country_code, rider_id ) as uid
 
FROM `fulfillment-dwh-production.cl.riders`

WHERE 
    ( DATE(created_at) BETWEEN '2020-04-01' AND '2020-04-30' )
AND country_code NOT LIKE '%dp%' 
AND rider_id IS NOT NULL
AND country_code = 'ar'
)

, rider_data AS (
SELECT
rider_list.country_code
, orders.city_id
, rider_list.uid
, d.vehicle.name as vehicle_name
, DATE(DATETIME( d.rider_dropped_off_at, d.timezone )) as order_date

FROM 
`fulfillment-dwh-production.cl.orders` as orders
CROSS JOIN UNNEST(orders.deliveries) as d

RIGHT JOIN
rider_list
 ON rider_list.country_code = orders.country_code
AND rider_list.rider_id = d.rider_id

WHERE
    (DATE(DATETIME( d.rider_dropped_off_at, d.timezone )) BETWEEN '2020-04-01' AND '2020-10-31')
AND orders.country_code NOT LIKE '%dp%'
AND d.rider_id IS NOT NULL
AND orders.country_code = 'ar'

GROUP BY 1,2,3,4,5
)

SELECT
report_date
, uid
, vehicle_name
, country_code
, city_id
, DATE_DIFF( report_date, MIN(order_date) , DAY ) as tenure
, DATE_DIFF( max(order_date), MIN(order_date) , DAY ) as recency
, COUNT( DISTINCT order_date ) -1 as frequency

FROM
report_dates
CROSS JOIN rider_data

WHERE order_date <= report_date

GROUP BY 1,2,3,4,5

HAVING DATE_DIFF( report_date, MIN(order_date) , DAY ) >= 0 
""".format(threshold_date=input_threshold_date, end_date=input_end_date)

df = pandas_gbq.read_gbq(sql, project_id=project_id)



Downloading: 100%|██████████| 218234/218234 [00:15<00:00, 14223.39rows/s]


In [ ]:
# save data to csv for future work without downloading again

df.to_csv('colab_churn_model_data_ar.csv')
!cp colab_churn_model_data_ar.csv "drive/My Drive/Colab Notebooks"

In [4]:
# retrieve prev downloaded csv data

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_churn_model_data_ar.csv')

In [5]:
# set up connection to gsheets using gspread


auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())




In [6]:
# divide data into cohorts
# vehicle, region, fill rate, city
# for now with country-specified, only split by vehicle?
# city_id to be relevant for determining fill rate?


# retrieve vehicle type groupings
vehicles_map_sheet = gc.open('Project_Rider_Churn_Birdhouse').worksheet('vehicles_map')

# get_all_values gives a list of lists
# get_all_records gives list of dicts
country_vehicle_dict = vehicles_map_sheet.get_all_records()

vehicles_map = pd.DataFrame.from_records(country_vehicle_dict) 

# create list of unique country_codes
countries_list = list(set(vehicles_map['country_code']))

# print(countries_list)
# print(vehicles_map)

In [ ]:
"""
eventually, what i want is:

{'ar': {'Auto': dataframes, 'Bici': dataframes2, 'Moto': dataframes3},
 'sg': {'Car': dataframes, 'Scooter': dataframes2, 'Bicycle': dataframes3}}
"""

In [7]:
# generate dict with hierarchy countrycode: {vehicle1: dataframes, vehicle2: dataframes2}
"""
e.g.
{'ar': {'Auto': dataframes, 'Bici': dataframes2, 'Moto': dataframes3},
 'sg': {'Car': dataframes, 'Scooter': dataframes2, 'Bicycle': dataframes3}}
"""

# create dict with keys only
countries_vehicles_dict = {country: None for country in countries_list}
# print(countries_vehicles_dict['ar'])


In [8]:
# populate dict with values

for country in vehicles_map['country_code']:
  if countries_vehicles_dict[country] is not None:
    countries_vehicles_dict[country].append(vehicles_map.loc[vehicles_map['country_code']==country, 'keys'])
  else:
    countries_vehicles_dict[country] = vehicles_map.loc[vehicles_map['country_code']==country, 'keys']

countries_vehicles_dict
# countries_vehicles_dict['ar']

{'ar': 0    Auto
 1    Bici
 2    Moto
 Name: keys, dtype: object, 'sg': 3    Scooter
 4    Bicycle
 5     Walker
 Name: keys, dtype: object}

In [9]:
# append dataframes to each country:key, e.g. 'ar': {'Bici': datraframes}

for country in countries_vehicles_dict:
  for vehicle in countries_vehicles_dict[country]:
    countries_vehicles_dict[country][vehicle] = df[(df['country_code']==country) & (df['vehicle_name'].str.contains(vehicle))]

# countries_vehicles_dict['ar']['Bici']

In [10]:
countries_vehicles_dict['ar']['Bici']

,Unnamed: 0,report_date,uid,vehicle_name,country_code,city_id,tenure,recency,frequency
0,0,2020-08-01,ar78577,Bicicleta,ar,2,91,91,82
1,1,2020-08-01,ar77459,Bicicleta,ar,1,99,99,78
2,2,2020-08-01,ar78471,Bicicleta,ar,4,86,80,53
3,3,2020-08-01,ar77156,Bicicleta,ar,1,93,78,30
4,4,2020-08-01,ar75673,Bicicleta,ar,1,89,33,18
...,...,...,...,...,...,...,...,...,...
218221,218221,2020-10-31,ar74334,Bicicleta,ar,4,192,155,49
218226,218226,2020-10-31,ar75065,Bicicleta,ar,1,185,66,8
218227,218227,2020-10-31,ar75539,Bicicleta,ar,206,182,8,4
218229,218229,2020-10-31,ar77674,Bicicleta,ar,201,175,55,16


In [ ]:
# sense check on ar74913
extracted = countries_vehicles_dict['ar']['Bici']
extracted = extracted[extracted['uid']=='ar74913']
extracted

,Unnamed: 0,report_date,uid,vehicle_name,country_code,city_id,tenure,recency,frequency
1715,1715,2020-08-01,ar74913,Bicicleta,ar,2,71,71,67
4359,4359,2020-08-02,ar74913,Bicicleta,ar,2,72,72,68
4987,4987,2020-08-03,ar74913,Bicicleta,ar,2,73,73,69
7027,7027,2020-08-04,ar74913,Bicicleta,ar,2,74,74,70
11029,11029,2020-08-05,ar74913,Bicicleta,ar,2,75,75,71
...,...,...,...,...,...,...,...,...,...
207267,207267,2020-10-27,ar74913,Bicicleta,ar,2,158,158,132
210548,210548,2020-10-28,ar74913,Bicicleta,ar,2,159,158,132
212400,212400,2020-10-29,ar74913,Bicicleta,ar,2,160,158,132
214567,214567,2020-10-30,ar74913,Bicicleta,ar,2,161,161,133


In [12]:
# generate list of dates, to iterate through for each day's predictions
# note that the report_date field, i.e. df['report_date'], is stored as datetime

list_report_date = list(set(df['report_date']))

In [13]:
# this is a test cell
# to find out what form time t should take, for
# conditional_probability_of_n_purchases_up_to_time(n, t, frequency, recency, tenure)

""" Parameters:	
    n (int) – number of purchases.
    t (a scalar) – time up to which probability should be calculated."""

test_date = list_report_date[30]
extracted_df = countries_vehicles_dict['ar']['Bici']
extracted_df_uidindexed = extracted_df.set_index('uid')
df_for_fitting = extracted_df_uidindexed[extracted_df_uidindexed['report_date']==test_date]

freq = df_for_fitting['frequency']
print('freq: \n\n',freq,'\n')

recency = df_for_fitting['recency']
print('recency: \n\n',recency,'\n')

tenure = df_for_fitting['tenure'] 
print('tenure: \n\n',tenure,'\n')   

PNBD = ParetoNBDFitter(penalizer_coef=0.0)
fitted_PNBD = PNBD.fit(freq,recency,tenure)
print('PNBD has been fitted!')

# test prediction for ar74913
n = (df_for_fitting.loc['ar74913', 'frequency'])+1
t = (df_for_fitting.loc['ar74913', 'tenure'])+30
frequency = df_for_fitting.loc['ar74913', 'frequency']
recency = df_for_fitting.loc['ar74913', 'recency']
tenure = df_for_fitting.loc['ar74913', 'tenure']

prediction = PNBD.conditional_probability_of_n_purchases_up_to_time(n, t, frequency, recency, tenure)
print(prediction)

freq: 

 uid
ar75060    104
ar76955     85
ar76837     22
ar77439     22
ar77691     76
          ... 
ar76439     92
ar78575     43
ar75756      6
ar74466      8
ar77562      0
Name: frequency, Length: 1224, dtype: int64 

recency: 

 uid
ar75060    108
ar76955    108
ar76837     68
ar77439     48
ar77691    114
          ... 
ar76439     99
ar78575     46
ar75756     16
ar74466     34
ar77562      0
Name: recency, Length: 1224, dtype: int64 

tenure: 

 uid
ar75060    121
ar76955    108
ar76837    103
ar77439    112
ar77691    114
          ... 
ar76439     99
ar78575     46
ar75756    117
ar74466     43
ar77562     84
Name: tenure, Length: 1224, dtype: int64 



/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


PNBD has been fitted!
inf


In [ ]:
# test prediction for ar74913
n = 1
t = 30
frequency = df_for_fitting.loc['ar74913', 'frequency']
recency = df_for_fitting.loc['ar74913', 'recency']
tenure = df_for_fitting.loc['ar74913', 'tenure']

print('ar74913 \n\n frequency:\n',frequency,'\n\n recency:\n',recency,'\n\n tenure:\n',tenure)

prediction = PNBD.conditional_probability_of_n_purchases_up_to_time(n, t, frequency, recency, tenure)
print(prediction)

ar74913 n
\ frequency:
 85 n
\ recency:
 96 n
\ tenure:
 96
0.005840958509474081


In [ ]:
# test prediction for ar74913
# this is the one to go for, for P(active on >0 days in next 30 days)
# following cell is an implemented variation of the same method
n = 1
t = 30
frequency = df_for_fitting.loc['ar74913', 'frequency']
recency = df_for_fitting.loc['ar74913', 'recency']
tenure = df_for_fitting.loc['ar74913', 'tenure']

print('ar74913 \n\n frequency:\n',frequency,'\n\n recency:\n',recency,'\n\n tenure:\n',tenure)

prediction = PNBD.conditional_expected_number_of_purchases_up_to_time(t, frequency, recency, tenure)
print(prediction/30)

ar74913 

 frequency:
 85 

 recency:
 96 

 tenure:
 96
0.8053766230052327


In [23]:
# test prediction for ar74913
# this is the one to go for, for P(active on >0 days in next 30 days)

test_date = list_report_date[30]
extracted_df = countries_vehicles_dict['ar']['Bici']
extracted_df_uidindexed = extracted_df.set_index('uid')
df_for_fitting = extracted_df_uidindexed[extracted_df_uidindexed['report_date']==test_date]

pred_inputs = df_for_fitting[[ 'frequency', 'recency', 'tenure']]
pred_inputs.insert(0, 'next_t_days', 30)

pred_inputs['p_active'] = pred_inputs.apply(lambda x: PNBD.conditional_expected_number_of_purchases_up_to_time(x['next_t_days'], x['frequency'], x['recency'], x['tenure']), axis=1)
pred_inputs['p_active'] = pred_inputs['p_active'].div(30).round(2)

pred_inputs


,next_t_days,frequency,recency,tenure,p_active
uid,,,,,
ar75060,30,104,108,121,0.00
ar76955,30,85,108,108,0.71
ar76837,30,22,68,103,0.00
ar77439,30,22,48,112,0.00
ar77691,30,76,114,114,0.61
...,...,...,...,...,...
ar76439,30,92,99,99,0.83
ar78575,30,43,46,46,0.82
ar75756,30,6,16,117,0.00


In [26]:
# testing if this will yield a DF for getting predictions

test_date = list_report_date[30]
extracted_df = countries_vehicles_dict['ar']['Bici']
extracted_df_uidindexed = extracted_df.set_index('uid')
df_for_fitting = extracted_df_uidindexed[extracted_df_uidindexed['report_date']==test_date]

pred_inputs = df_for_fitting[[ 'frequency', 'recency', 'tenure']]
pred_inputs.insert(0, 'n', 30)

pred_inputs

df_for_fitting

,Unnamed: 0,report_date,vehicle_name,country_code,city_id,tenure,recency,frequency
uid,,,,,,,,
ar75060,47290,2020-08-22,Bicicleta,ar,1,121,108,104
ar76955,47291,2020-08-22,Bicicleta,ar,203,108,108,85
ar76837,47292,2020-08-22,Bicicleta,ar,1,103,68,22
ar77439,47294,2020-08-22,Bicicleta,ar,1,112,48,22
ar77691,47300,2020-08-22,Bicicleta,ar,1,114,114,76
...,...,...,...,...,...,...,...,...
ar76439,49562,2020-08-22,CAFE - Bici,ar,1,99,99,92
ar78575,49564,2020-08-22,CAFE - Bici,ar,1,46,46,43
ar75756,49565,2020-08-22,Bicicleta,ar,204,117,16,6


In [ ]:
### implement a for loop where 1 model is fitted for each country-vehicle, and all these models saved in a list
### the list, containing fitted models, must have associated features that help future code identify which model to pick


# eventually create fitted models as pkl (pickle) objects with naming convention countrycode_vehicle
# actually, no, don't do this, just fit day-by-day
# countrycode derived from countries_vehicles_dict[i] and PNBD_countries_vehicles_dict[i][a]

uid_predictions = pd.DataFrame()

for country in countries_vehicles_dict:
  for vehicle in countries_vehicles_dict[country]:
    if countries_vehicles_dict[country][vehicle] is None:
      break
    else:
      extracted_df = countries_vehicles_dict[country][vehicle] # retrieve data for each country+vehicle
      extracted_df_uidindexed = extracted_df.set_index('uid') # convert the index to uid

      for date in list_report_date:
        df_for_fitting = extracted_df_uidindexed[extracted_df_uidindexed['report_date']==date]

        freq = df_for_fitting['frequency']
        recency = df_for_fitting['recency']
        tenure = df_for_fitting['tenure']

        PNBD = ParetoNBDFitter(penalizer_coef=0.0)
        fitted_PNBD = PNBD.fit(freq,recency,tenure)

        # x = 30 # for P(active on >0 days in next x days)
        # create DF of model's input values in format (x,frequency,recency,tenure) 
        # is it possible to use .apply with conditional_expected_number_of_purchases_up_to_time, passing a Series containing all parameters?
        # pred_inputs = df_for_fitting[['frequency', 'recency', 'tenure']]
        # pred_inputs.insert(0, 'n', 30)
        """the block above is a longer variation replaced by a simpler version, but using the same method"""

        next_t_days = 30 # for P(active on >0 days in next t days)
        df_for_pred = df_for_fitting.copy()
        df_for_pred['next_t_days'] = next_t_days # create new col containing next t days params

        # prediction and saving of prediction values happens here
        # create new col 'p_active' in df_for_pred, assign prediction values here
        # and use .apply( lambda x: ) to make predictions 
        # divide expected no. purchases by next_t_days to get actual p_active, and round to 2dp
        df_for_pred['p_active'] = df_for_pred.apply(lambda x: PNBD.conditional_expected_number_of_purchases_up_to_time(x['next_t_days'], x['frequency'], x['recency'], x['tenure']), axis=1)
        df_for_pred['p_active'] = df_for_pred['p_active'].div(next_t_days).round(2)

        # add uid and report_date to df_for_pred
        df_for_pred['uid'] = df_for_pred.index
        df_for_pred['report_date'] = date

        # save uid, report_date, cohorting info, pred inputs and p_active in uid_predictions
        uid_predictions = uid_predictions.append(df_for_pred[['uid', 'report_date', 'country_code', 'vehicle_name', 'frequency', 'recency', 'tenure', 'p_active']])
        print('predictions made and saved for {} {} [}'.format(date, country, vehicle))

print('uid_predictions complete!')

uid_predictions.to_csv('/content/drive/My Drive/Colab Notebooks/colab_churn_model_ar_result.csv')

      





TypeError: ignored

In [ ]:
for rider, data in enumerate(df_for_fitting[:10]):
  print( rider, data)

print(df_for_fitting[:10])

0 Unnamed: 0
1 report_date
2 vehicle_name
3 country_code
4 city_id
5 tenure
6 recency
7 frequency
         Unnamed: 0 report_date vehicle_name  ... tenure  recency  frequency
uid                                           ...                           
ar76588       56471  2020-08-26    Bicicleta  ...    107      107         90
ar77953       56472  2020-08-26    Bicicleta  ...    117      117        111
ar75109       56475  2020-08-26    Bicicleta  ...     78       78         66
ar78016       56477  2020-08-26    Bicicleta  ...    107      107         43
ar76265       56482  2020-08-26    Bicicleta  ...    117      114         68
ar74832       56488  2020-08-26    Bicicleta  ...    139       36         35
ar75674       56490  2020-08-26    Bicicleta  ...    124      124        122
ar76122       56492  2020-08-26    Bicicleta  ...    128      128         95
ar74923       56493  2020-08-26    Bicicleta  ...    138      112         80
ar75712       56494  2020-08-26    Bicicleta  ...    11

In [ ]:
# convert the index to uid
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
# note - this should be done only just before the training phase, when the DF has been extracted from the dict

df = df.set_index('uid')
df

,report_date,vehicle_name,country_code,city_id,tenure,recency,frequency
uid,,,,,,,
ar78156,2020-08-01,Motocicleta,ar,200,85,176,7
ar78442,2020-08-01,Motocicleta,ar,200,91,148,15
ar77054,2020-08-01,Bicicleta,ar,228,9,0,0
ar77562,2020-08-01,Bicicleta,ar,223,82,0,0
ar75197,2020-08-01,Motocicleta,ar,1,92,61,0
...,...,...,...,...,...,...,...
ar77296,2020-10-31,Bicicleta,ar,1,183,108,0
ar78612,2020-10-31,Bicicleta,ar,3,176,81,0
ar74323,2020-10-31,Bicicleta,ar,208,187,187,5
